<a href="https://colab.research.google.com/github/GowthamNats/Recipe-Recommender-System/blob/main/SNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello World")

Hello World


In [ ]:
import numpy as np
import pandas as pd

import os
df = pd.read_json('../content/test.json')
df.ingredients = df.ingredients.apply(lambda ings : [ing.lower() for ing in ings])
df

,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."
3,29752,"[meat cuts, file powder, smoked sausage, okra,..."
4,35687,"[ground black pepper, salt, sausage casings, l..."
...,...,...
9939,30246,"[large egg yolks, fresh lemon juice, sugar, bo..."
9940,36028,"[hot sauce, butter, sweet potatoes, adobo sauc..."
9941,22339,"[black pepper, salt, parmigiano reggiano chees..."
9942,42525,"[cheddar cheese, cayenne, paprika, plum tomato..."


In [ ]:
import itertools
from collections import Counter
cooc_counts = Counter()
ing_count  = Counter()
for ingredients in df.ingredients:
    for ing in ingredients:
        ing_count[ing] += 1
    for (ing_a, ing_b) in itertools.combinations(set(ingredients), 2):
       if ing_a > ing_b:
            ing_a, ing_b = ing_b, ing_a
       cooc_counts[(ing_a, ing_b)] += 1

In [ ]:
cooc_df = pd.DataFrame(((ing_a, ing_b, ing_count[ing_a], ing_count[ing_b], cooc) for (ing_a, ing_b), cooc in cooc_counts.items()), columns=['a', 'b', 'a_count', 'b_count', 'cooc'])
cooc_df

,a,b,a_count,b_count,cooc
0,all-purpose flour,white sugar,1184,299,103
1,all-purpose flour,milk,1184,588,211
2,all-purpose flour,raisins,1184,135,29
3,all-purpose flour,eggs,1184,874,229
4,all-purpose flour,baking powder,1184,430,286
...,...,...,...,...,...
206259,boiling water,yukon gold potatoes,89,47,1
206260,cilantro sprigs,tomato salsa,64,11,1
206261,boiling water,tomato salsa,89,11,1
206262,tomato salsa,tomatoes,11,754,1


In [ ]:
cooc_df[cooc_df.a == 'chillies'].sort_values('cooc', ascending=False).head(10)

,a,b,a_count,b_count,cooc
5869,chillies,onions,27,2036,15
5872,chillies,salt,27,4485,13
5862,chillies,garam masala,27,254,9
58748,chillies,garlic,27,1791,9
5871,chillies,vegetable oil,27,1131,8
5845,chillies,cumin seed,27,254,7
104671,chillies,ginger,27,435,7
26521,chillies,fresh coriander,27,73,7
5874,chillies,tomatoes,27,754,7
5868,chillies,tumeric,27,195,7


In [ ]:
p_a = cooc_df.a_count / sum(ing_count.values())
p_b = cooc_df.b_count / len(ing_count.values())
p_a_b = cooc_df.cooc / cooc_df.cooc.sum()
cooc_df['pmi'] = np.log(p_a_b / (p_a * p_b))
cooc_df.sort_values('pmi', ascending=False).head(10)

,a,b,a_count,b_count,cooc,pmi
164575,dende oil,rock shrimp,1,1,1,6.645209
100612,cherry pie filling,cola,1,1,1,6.645209
164361,"english muffins, split and toasted",turkey gravy,1,1,1,6.645209
188780,spice islands garlic powder,spice islands® chipotle chile powder,1,1,1,6.645209
54008,dried cloud ears,lily buds,1,1,1,6.645209
80962,capicola,italian sandwich rolls,1,1,1,6.645209
40152,potassium carbonate,shrimp shells,1,1,1,6.645209
181649,raw milk,rennet,1,1,1,6.645209
80996,malbec,smoked sea salt,1,1,1,6.645209
145657,marrow bones,veal bones,1,1,1,6.645209


In [ ]:
min_count = 5
cooc_df[(cooc_df.a_count >= min_count) & (cooc_df.b_count >= min_count)].sort_values('pmi', ascending=False).head(20)

,a,b,a_count,b_count,cooc,pmi
160791,açai,granola,5,5,3,4.524945
48351,frozen whipped topping,yellow cake mix,5,5,2,4.119480
101611,chia seeds,granola,5,5,2,4.119480
160782,açai,goji berries,5,5,2,4.119480
656,glace cherries,sultana,5,8,3,4.054941
43693,pepperoncini,roast beef,5,6,2,3.937158
703,glace cherries,mixed spice,5,9,3,3.937158
48359,frozen whipped topping,vanilla instant pudding,5,6,2,3.937158
76388,asafoetida powder,split black lentils,12,7,5,3.823830
145024,nam pla,yardlong beans,6,6,2,3.754837


In [ ]:
min_count = 30
cooc_df[(cooc_df.a_count >= min_count) & (cooc_df.b_count >= min_count)].sort_values('pmi', ascending=False).head(20)

,a,b,a_count,b_count,cooc,pmi
20302,garlic paste,ginger paste,78,35,23,1.868646
101541,gochujang base,kimchi,36,32,8,1.675395
3209,kaffir lime leaves,lemongrass,35,85,19,1.591648
22036,chili oil,szechwan peppercorns,31,36,7,1.573613
15608,coriander powder,garlic paste,56,78,26,1.521245
24440,garlic paste,red chili powder,78,57,26,1.503545
24508,paneer,red chili powder,36,57,12,1.503545
44018,parsley sprigs,thyme sprigs,36,39,8,1.477569
24669,coriander powder,red chili powder,56,57,18,1.467177
24525,coriander powder,paneer,56,36,11,1.434233


In [ ]:
min_count = 30
cooc_df[(cooc_df.a_count >= min_count) & (cooc_df.b_count >= min_count) & (cooc_df.cooc > 1)].sort_values('pmi', ascending=True).head(20)

,a,b,a_count,b_count,cooc,pmi
42137,garlic cloves,white sugar,1535,299,2,-5.698374
5259,fine sea salt,salt,78,4485,2,-5.426847
86070,grated parmesan cheese,ground cumin,481,684,2,-5.365469
20523,garlic cloves,minced garlic,1535,418,4,-5.340264
54888,oil,unsalted butter,456,692,2,-5.323723
41561,sesame oil,tomatoes,410,754,2,-5.303194
41697,cooking spray,soy sauce,364,824,2,-5.272969
30060,salsa,sugar,258,1630,3,-5.205474
94365,all-purpose flour,fresh lime juice,1184,346,3,-5.179272
34416,garlic,large garlic cloves,1791,223,3,-5.153880


In [ ]:
from scipy.sparse import csr_matrix
data_df = cooc_df[cooc_df.pmi > 0].copy()
# Since the matrix is symetric, we add the same values for (b,a) as we have for (a,b)
data_df_t = data_df.copy()
data_df.a, data_df.b = data_df.b, data_df.a
data_df = pd.concat([data_df, data_df_t])

rows_idx, row_keys = pd.factorize(data_df.a)
cols_idx, col_keys = pd.factorize(data_df.b)
values = data_df.pmi

matrix = csr_matrix((values, (rows_idx, cols_idx)))
key_to_row = {key: idx for idx, key in enumerate(row_keys)}

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(200)
factors = svd.fit_transform(matrix)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def most_similar(ingredient, topn=10):
    if ingredient not in key_to_row:
        print("Unknown ingredient.")
    factor = factors[key_to_row[ingredient]]
    cosines = cosine_similarity([factor], factors)[0]
    indices = cosines.argsort()[::-1][:topn + 1]
    keys = [row_keys[idx] for idx in indices if idx != key_to_row[ingredient]]
    return keys, cosines[indices]

def display_most_similar(ingredient, topn=10):
    print("- Most similar to '{}'".format(ingredient))
    for similar_ing, score in zip(*most_similar(ingredient, topn)):
        print("  . {} : {:.2f}".format(similar_ing, score)) 

In [ ]:
display_most_similar('chile powder')

- Most similar to 'chile powder'
  . fully cooked luncheon meat : 1.00
  . sliced cucumber : 0.75
  . sliced turkey : 0.71
  . marjoram : 0.71
  . lite coconut milk : 0.71
  . kim chee : 0.70
  . greek seasoning : 0.70
  . feta cheese : 0.70
  . baby portobello mushrooms : 0.68
  . roast red peppers, drain : 0.68
